In [1]:
!pip install -q catboost
!pip install -q pycaret
!pip install -q optuna
!pip install -q pycaret[tuners]
!pip uninstall packaging
!pip install packaging
!pip install rdkit-pypi -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.4/484.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.2 MB/s eta 0:

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as spst
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,classification_report,mean_absolute_percentage_error,mean_squared_error
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from sklearn.impute import KNNImputer
from catboost import CatBoostRegressor
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_useSVG=True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [49]:
train = pd.read_csv("/content/drive/MyDrive/dacon/ccc/train.csv")
test = pd.read_csv("/content/drive/MyDrive/dacon/ccc/test.csv")
sample_submission = pd.read_csv("/content/drive/MyDrive/dacon/ccc/sample_submission.csv")

In [50]:
from rdkit.Chem import Descriptors
def number_of_atoms(atom_list, df):
    for i in atom_list:
        df['num_of_{}_atoms'.format(i)] = df['mol'].apply(lambda x: len(x.GetSubstructMatches(Chem.MolFromSmiles(i))))

train['mol'] = train['SMILES'].apply(lambda x: Chem.MolFromSmiles(x))

train['mol'] = train['mol'].apply(lambda x: Chem.AddHs(x))
train['num_of_atoms'] = train['mol'].apply(lambda x: x.GetNumAtoms())
train['num_of_heavy_atoms'] = train['mol'].apply(lambda x: x.GetNumHeavyAtoms())

number_of_atoms(['C','O', 'N', 'Cl'], train)

train['tpsa'] = train['mol'].apply(lambda x: Descriptors.TPSA(x))
train['mol_w'] = train['mol'].apply(lambda x: Descriptors.ExactMolWt(x))
train['num_valence_electrons'] = train['mol'].apply(lambda x: Descriptors.NumValenceElectrons(x))
train['num_heteroatoms'] = train['mol'].apply(lambda x: Descriptors.NumHeteroatoms(x))

In [51]:
train_mlm = train.drop('HLM',axis=1)
train_hlm = train.drop('MLM',axis=1)

In [52]:
train_mlm = train_mlm.drop(['id','SMILES','mol'],axis=1)
train_hlm = train_hlm.drop(['id','SMILES','mol'],axis=1)

In [35]:
train_hlm

,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,num_of_atoms,num_of_heavy_atoms,num_of_C_atoms,num_of_O_atoms,num_of_N_atoms,num_of_Cl_atoms,tpsa,mol_w,num_valence_electrons,num_heteroatoms
0,50.680,3.259,400.495,5,2,8,3.259,117.37,52,28,20,3,4,0,89.13,400.156912,148,8
1,50.590,2.169,301.407,2,1,2,2.172,73.47,40,21,16,1,3,0,45.23,301.124883,110,5
2,80.892,1.593,297.358,5,0,3,1.585,62.45,41,22,15,0,7,0,62.45,297.170194,114,7
3,2.000,4.771,494.652,6,0,5,3.475,92.60,69,35,26,2,6,0,84.22,494.246395,186,9
4,99.990,2.335,268.310,3,0,1,2.337,42.43,36,20,16,2,2,0,42.43,268.121178,102,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,3.079,3.409,396.195,3,1,5,3.409,64.74,39,25,14,1,5,2,64.74,395.052750,136,11
3494,47.630,1.912,359.381,4,1,3,1.844,77.37,44,27,20,2,5,0,85.04,359.138225,134,7
3495,1.790,1.941,261.320,3,1,6,2.124,70.14,38,19,14,2,3,0,70.14,261.147727,102,5
3496,2.770,0.989,284.696,5,1,5,0.989,91.51,32,19,12,4,2,1,91.51,284.056385,102,7


In [37]:
train_mlm.corr()

,MLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,num_of_atoms,num_of_heavy_atoms,num_of_C_atoms,num_of_O_atoms,num_of_N_atoms,num_of_Cl_atoms,tpsa,mol_w,num_valence_electrons,num_heteroatoms
MLM,1.000000,-0.330083,-0.081239,0.164510,0.210836,-0.092563,-0.350146,0.184850,-0.123796,-0.095937,-0.181404,0.093109,0.113832,-0.015845,0.245155,-0.083254,-0.085244,0.164927
AlogP,-0.330083,1.000000,0.389760,-0.284415,-0.172222,0.111844,0.957611,-0.298194,0.337656,0.434591,0.581333,-0.165906,-0.166344,0.220627,-0.316789,0.418142,0.369519,-0.186592
Molecular_Weight,-0.081239,0.389760,1.000000,0.471814,0.116186,0.583711,0.369462,0.439114,0.812382,0.925911,0.806930,0.438139,0.143016,0.122444,0.401361,0.982454,0.930898,0.607325
Num_H_Acceptors,0.164510,-0.284415,0.471814,1.000000,0.208433,0.474012,-0.305506,0.714315,0.344610,0.425698,0.193494,0.616088,0.256363,-0.083929,0.721510,0.435532,0.459190,0.692786
Num_H_Donors,0.210836,-0.172222,0.116186,0.208433,1.000000,0.176871,-0.212082,0.474614,0.065733,0.079056,0.002002,0.172267,0.109412,-0.044154,0.503915,0.077620,0.093809,0.207713
Num_RotatableBonds,-0.092563,0.111844,0.583711,0.474012,0.176871,1.000000,0.071659,0.371574,0.623724,0.543946,0.454143,0.449216,0.015640,-0.029788,0.355177,0.557225,0.610484,0.403165
LogD,-0.350146,0.957611,0.369462,-0.305506,-0.212082,0.071659,1.000000,-0.294670,0.300547,0.415669,0.552059,-0.177037,-0.138197,0.221319,-0.308616,0.399324,0.345191,-0.169580
Molecular_PolarSurfaceArea,0.184850,-0.298194,0.439114,0.714315,0.474614,0.371574,-0.294670,1.000000,0.218501,0.346908,0.070849,0.508521,0.306242,-0.115654,0.851826,0.409947,0.372487,0.766873
num_of_atoms,-0.123796,0.337656,0.812382,0.344610,0.065733,0.623724,0.300547,0.218501,1.000000,0.895886,0.885084,0.384241,0.118157,-0.109753,0.273338,0.823041,0.939730,0.344575
num_of_heavy_atoms,-0.095937,0.434591,0.925911,0.425698,0.079056,0.543946,0.415669,0.346908,0.895886,1.000000,0.930271,0.411884,0.200424,0.002502,0.404575,0.942616,0.984617,0.518988


In [38]:
train_hlm.corr()

,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,num_of_atoms,num_of_heavy_atoms,num_of_C_atoms,num_of_O_atoms,num_of_N_atoms,num_of_Cl_atoms,tpsa,mol_w,num_valence_electrons,num_heteroatoms
HLM,1.000000,-0.346022,-0.175117,0.092313,0.176549,-0.132263,-0.357456,0.094323,-0.169343,-0.173623,-0.232584,0.006772,0.123738,-0.028590,0.163990,-0.177041,-0.162056,0.075471
AlogP,-0.346022,1.000000,0.389760,-0.284415,-0.172222,0.111844,0.957611,-0.298194,0.337656,0.434591,0.581333,-0.165906,-0.166344,0.220627,-0.316789,0.418142,0.369519,-0.186592
Molecular_Weight,-0.175117,0.389760,1.000000,0.471814,0.116186,0.583711,0.369462,0.439114,0.812382,0.925911,0.806930,0.438139,0.143016,0.122444,0.401361,0.982454,0.930898,0.607325
Num_H_Acceptors,0.092313,-0.284415,0.471814,1.000000,0.208433,0.474012,-0.305506,0.714315,0.344610,0.425698,0.193494,0.616088,0.256363,-0.083929,0.721510,0.435532,0.459190,0.692786
Num_H_Donors,0.176549,-0.172222,0.116186,0.208433,1.000000,0.176871,-0.212082,0.474614,0.065733,0.079056,0.002002,0.172267,0.109412,-0.044154,0.503915,0.077620,0.093809,0.207713
Num_RotatableBonds,-0.132263,0.111844,0.583711,0.474012,0.176871,1.000000,0.071659,0.371574,0.623724,0.543946,0.454143,0.449216,0.015640,-0.029788,0.355177,0.557225,0.610484,0.403165
LogD,-0.357456,0.957611,0.369462,-0.305506,-0.212082,0.071659,1.000000,-0.294670,0.300547,0.415669,0.552059,-0.177037,-0.138197,0.221319,-0.308616,0.399324,0.345191,-0.169580
Molecular_PolarSurfaceArea,0.094323,-0.298194,0.439114,0.714315,0.474614,0.371574,-0.294670,1.000000,0.218501,0.346908,0.070849,0.508521,0.306242,-0.115654,0.851826,0.409947,0.372487,0.766873
num_of_atoms,-0.169343,0.337656,0.812382,0.344610,0.065733,0.623724,0.300547,0.218501,1.000000,0.895886,0.885084,0.384241,0.118157,-0.109753,0.273338,0.823041,0.939730,0.344575
num_of_heavy_atoms,-0.173623,0.434591,0.925911,0.425698,0.079056,0.543946,0.415669,0.346908,0.895886,1.000000,0.930271,0.411884,0.200424,0.002502,0.404575,0.942616,0.984617,0.518988


In [53]:
x_mlm = train_mlm.drop('MLM',axis=1)
y_mlm = train_mlm[['MLM']]

x_hlm = train_hlm.drop('HLM',axis=1)
y_hlm = train_hlm[['HLM']]

x_train_mlm,x_test_mlm,y_train_mlm,y_test_mlm = train_test_split(x_mlm,y_mlm,test_size=0.2,random_state=42)
x_train_hlm,x_test_hlm,y_train_hlm,y_test_hlm = train_test_split(x_hlm,y_hlm,test_size=0.2,random_state=42)

model = CatBoostRegressor(random_state=42)
model.fit(x_train_mlm,y_train_mlm)
y_pred = model.predict(x_test_mlm)
print(np.sqrt(mean_squared_error(y_test_mlm,y_pred)))

Learning rate set to 0.048171
0:	learn: 35.1751984	total: 2.67ms	remaining: 2.67s
1:	learn: 34.8554380	total: 5.08ms	remaining: 2.53s
2:	learn: 34.5968136	total: 7.38ms	remaining: 2.45s
3:	learn: 34.3600382	total: 9.53ms	remaining: 2.37s
4:	learn: 34.1530195	total: 11.7ms	remaining: 2.33s
5:	learn: 33.9351510	total: 15.6ms	remaining: 2.58s
6:	learn: 33.7339265	total: 17.9ms	remaining: 2.54s
7:	learn: 33.5554602	total: 20.4ms	remaining: 2.52s
8:	learn: 33.3761140	total: 22.5ms	remaining: 2.48s
9:	learn: 33.2124470	total: 25.1ms	remaining: 2.48s
10:	learn: 33.0534567	total: 27.4ms	remaining: 2.46s
11:	learn: 32.9127190	total: 29.6ms	remaining: 2.44s
12:	learn: 32.7709738	total: 31.7ms	remaining: 2.41s
13:	learn: 32.6417924	total: 34.1ms	remaining: 2.4s
14:	learn: 32.5246501	total: 36.4ms	remaining: 2.39s
15:	learn: 32.4203176	total: 38.5ms	remaining: 2.37s
16:	learn: 32.3129673	total: 40.8ms	remaining: 2.36s
17:	learn: 32.2168745	total: 43ms	remaining: 2.35s
18:	learn: 32.1210482	total: 

In [54]:
model = CatBoostRegressor(random_state=42)
model.fit(x_train_hlm,y_train_hlm)
y_pred = model.predict(x_test_hlm)
print(np.sqrt(mean_squared_error(y_test_hlm,y_pred)))

Learning rate set to 0.048171
0:	learn: 35.8477267	total: 2.37ms	remaining: 2.37s
1:	learn: 35.5996581	total: 5.01ms	remaining: 2.5s
2:	learn: 35.3777807	total: 7.37ms	remaining: 2.45s
3:	learn: 35.1535822	total: 10.2ms	remaining: 2.53s
4:	learn: 34.9622401	total: 16.6ms	remaining: 3.31s
5:	learn: 34.7377163	total: 19.7ms	remaining: 3.27s
6:	learn: 34.5314076	total: 23.2ms	remaining: 3.28s
7:	learn: 34.3413145	total: 27.1ms	remaining: 3.36s
8:	learn: 34.1624545	total: 31.5ms	remaining: 3.46s
9:	learn: 34.0002433	total: 33.8ms	remaining: 3.35s
10:	learn: 33.8315087	total: 36.2ms	remaining: 3.26s
11:	learn: 33.6996010	total: 38.8ms	remaining: 3.19s
12:	learn: 33.5646668	total: 41.5ms	remaining: 3.15s
13:	learn: 33.4465503	total: 43.9ms	remaining: 3.09s
14:	learn: 33.3356865	total: 47ms	remaining: 3.08s
15:	learn: 33.2356194	total: 49.7ms	remaining: 3.06s
16:	learn: 33.1515217	total: 52.1ms	remaining: 3.01s
17:	learn: 33.0703390	total: 54.4ms	remaining: 2.97s
18:	learn: 32.9736678	total: 

In [55]:
from rdkit.Chem import Descriptors
def number_of_atoms(atom_list, df):
    for i in atom_list:
        df['num_of_{}_atoms'.format(i)] = df['mol'].apply(lambda x: len(x.GetSubstructMatches(Chem.MolFromSmiles(i))))

test['mol'] = test['SMILES'].apply(lambda x: Chem.MolFromSmiles(x))

test['mol'] = test['mol'].apply(lambda x: Chem.AddHs(x))
test['num_of_atoms'] = test['mol'].apply(lambda x: x.GetNumAtoms())
test['num_of_heavy_atoms'] = test['mol'].apply(lambda x: x.GetNumHeavyAtoms())

number_of_atoms(['C','O', 'N', 'Cl'], test)

test['tpsa'] = test['mol'].apply(lambda x: Descriptors.TPSA(x))
test['mol_w'] = test['mol'].apply(lambda x: Descriptors.ExactMolWt(x))
test['num_valence_electrons'] = test['mol'].apply(lambda x: Descriptors.NumValenceElectrons(x))
test['num_heteroatoms'] = test['mol'].apply(lambda x: Descriptors.NumHeteroatoms(x))
test = test.drop(['id','SMILES','mol'],axis=1)

test

,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,num_of_atoms,num_of_heavy_atoms,num_of_C_atoms,num_of_O_atoms,num_of_N_atoms,num_of_Cl_atoms,tpsa,mol_w,num_valence_electrons,num_heteroatoms
0,2.641,361.505,4,2,7,2.635,92.76,52,25,18,1,5,0,64.52,361.193631,136,7
1,0.585,370.399,5,0,3,0.585,68.31,49,27,20,5,2,0,70.00,370.152872,142,7
2,4.276,347.414,4,4,5,4.290,92.86,47,26,20,1,5,0,97.15,347.174610,132,6
3,1.795,345.358,5,0,2,1.795,81.21,41,26,18,1,7,0,81.21,345.133808,128,8
4,1.219,353.418,4,0,2,0.169,61.15,49,26,19,2,5,0,62.85,353.185175,136,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,4.207,306.443,2,1,7,4.207,55.13,52,22,18,2,2,0,55.13,306.230728,124,4
486,-0.608,335.398,5,0,1,-1.736,70.16,49,24,17,4,3,0,70.16,335.184506,132,7
487,1.792,349.383,3,1,3,1.792,69.72,45,26,20,3,3,0,69.72,349.142641,132,6
488,0.790,341.132,3,2,2,0.423,69.64,30,20,13,3,2,0,71.33,339.985883,104,7


In [73]:
model_mlm = CatBoostRegressor(random_state=42)
model_mlm.fit(x_mlm,y_mlm)
y_pred_mlm = model_mlm.predict(test)


model_hml = CatBoostRegressor(random_state=42)
model_hml.fit(x_hlm,y_hlm)
y_pred_hlm = model_hml.predict(test)

Learning rate set to 0.0499
0:	learn: 35.3909810	total: 3.19ms	remaining: 3.19s
1:	learn: 35.1013237	total: 7.85ms	remaining: 3.92s
2:	learn: 34.8495376	total: 10.2ms	remaining: 3.37s
3:	learn: 34.6064618	total: 13.6ms	remaining: 3.38s
4:	learn: 34.3964305	total: 16.5ms	remaining: 3.29s
5:	learn: 34.1793670	total: 18.9ms	remaining: 3.14s
6:	learn: 34.0012043	total: 21.4ms	remaining: 3.04s
7:	learn: 33.8156714	total: 23.9ms	remaining: 2.96s
8:	learn: 33.6369024	total: 26.5ms	remaining: 2.92s
9:	learn: 33.4532518	total: 29.2ms	remaining: 2.89s
10:	learn: 33.3030570	total: 31.8ms	remaining: 2.86s
11:	learn: 33.1898702	total: 36.5ms	remaining: 3s
12:	learn: 33.0574108	total: 39.2ms	remaining: 2.98s
13:	learn: 32.9392752	total: 41.8ms	remaining: 2.94s
14:	learn: 32.8182284	total: 44.3ms	remaining: 2.91s
15:	learn: 32.7176232	total: 46.9ms	remaining: 2.88s
16:	learn: 32.6161002	total: 49.4ms	remaining: 2.86s
17:	learn: 32.5465603	total: 51.8ms	remaining: 2.82s
18:	learn: 32.4565615	total: 55

In [74]:
sample_submission['MLM'] = y_pred_mlm
sample_submission

,id,MLM,HLM
0,TEST_000,14.756320,42.692799
1,TEST_001,51.318463,66.815720
2,TEST_002,25.914637,52.230108
3,TEST_003,62.616508,81.863536
4,TEST_004,65.397273,89.269448
...,...,...,...
485,TEST_485,-4.884061,13.906585
486,TEST_486,88.134644,90.607999
487,TEST_487,47.300669,72.724668
488,TEST_488,58.462504,78.981385


In [75]:
sample_submission['HLM'] = y_pred_hlm

In [76]:
sample_submission

,id,MLM,HLM
0,TEST_000,14.756320,44.420415
1,TEST_001,51.318463,61.737252
2,TEST_002,25.914637,52.665390
3,TEST_003,62.616508,79.263590
4,TEST_004,65.397273,81.673367
...,...,...,...
485,TEST_485,-4.884061,18.556621
486,TEST_486,88.134644,93.946188
487,TEST_487,47.300669,69.486153
488,TEST_488,58.462504,61.748976


In [77]:
sample_submission.to_csv('test_catreg1.csv',index=False)